In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


from animalshelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Logo import
                               
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Image layout and anchor
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                height = 250, width = 250))] href = 'https://www.snhu.edu',target = "_blank"),
    html.Center(html.B(html.H1('Michelle Dumas - AAC DashBoard'))),
    html.Hr(),

    #Interactive filters
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
        ],
        value='All'

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),

        #Dashboard layout 
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
                             
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

#CRUD module calls when filters are selected
def update_dashboard(value):
    if (value == 'All'):
        df = pd.DataFrame.from_records(shelter.read({}))
    elif (value == 'Water'):
        df = pd.DataFrame(list(shelter.read(
            {
                "breed" : {
                    "$in": [
                        "Labrador Retriever Mix", 
                        "Chesapeake Bay Retriever", 
                        "Newfoundland"]}, 
                "sex_upon_outcome" : "Intact Female", 
                "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}
            })))
    elif (value == 'Mountain'):
        df = pd.DataFrame(list(shelter.read(
            {
                "breed" : {
                    "$in": [
                        "German Shephard", 
                        "Alaskan Malamute", 
                        "Old English Sheepdog", 
                        "Siberian Husky", 
                        "Rottweiler"]}, 
                "sex_upon_outcome" : "Intact Male", 
                "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}
            })))
    elif (value == 'Disaster'):
        df = pd.DataFrame(list(shelter.read(
            {
                "breed" : {
                    "$in": [
                        "Doberman Pinscher", 
                        "German Shephard", 
                        "Golden Retriever", 
                        "Bloodhound", 
                        "Rottweiler"]}, 
                "sex_upon_outcome" : "Intact Male", 
                "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}
            })))

        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed',)
        )    
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_selected_rows")])


#Geolocation map
def update_map(virtualRows):
                                                        
    if not virtualRows: 
        markerArray = (30.75,-97.48)
        toolTip = "Austin Animal Center"
        heading = "Austin Animal Center"
        description = "Shelter Location"
        
    #retriving lat and long from database and pushing it to map
    else: 
        dff = pd.DataFrame(df.iloc[virtualRows]) 
        lat = float(dff['location_lat'].to_string().split()[1]) 
        long = float(dff['location_long'].to_string().split()[1]) 
        markerArray = (lat, long) 

        toolTip = dff['breed']
        heading = "Animal Name"
        description = dff['name']
    
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=markerArray, children=[
                dl.Tooltip(toolTip),
                dl.Popup([
                    html.H1(heading),
                    html.P(description)
                ])
            ])
        ])
    ]                     



app

Connection successful
